In [1]:
from esa import SAW
import pandas as pd
CPATH = r'C:\Users\maxbr\OneDrive - Texas A&M University\Grad\PowerWorld\ACTIVSg2000\ACTIVSg2000.PWB'
saw = SAW(CPATH)

This code is setting V/Hz definite time relays that trips at 110% after 60s and at 118% after 2-6 seconds. 
Following IEEE Std. C37.106-2003

In [2]:
relay_kf = saw.get_key_field_list('RelayModel_GVPHZFT')
otherfields = ['Single', 'Single:1', 'Single:2', 'Single:3', 'Single:4']
'''
    Single - VHZpickup1 -> First V/Hz Setpoint
    Single:1 - tdelay1 -> Pick-up Time
    Single:2 - VHZpickup2 -> Second V/Hz Setpoint
    Single:3 - tdelay2 -> Pick-up Time
    Single:4 - bdelay -> Breaker Open Time
'''
VPHZ = saw.GetParametersMultipleElement('RelayModel_GVPHZFT', relay_kf + otherfields)
VPHZ

,BusNum,GenID,Single,Single:1,Single:2,Single:3,Single:4
0,1004,1,1.1,60,1.18,3,0.15
1,1006,1,1.1,60,1.18,3,0.15
2,1009,1,1.1,60,1.18,3,0.15
3,1011,1,1.1,60,1.18,3,0.15
4,1021,1,1.1,60,1.18,3,0.15
...,...,...,...,...,...,...,...
539,8152,1,1.1,60,1.18,3,0.15
540,8155,1,1.1,60,1.18,3,0.15
541,8155,2,1.1,60,1.18,3,0.15
542,8158,1,1.1,60,1.18,3,0.15


In [3]:
if VPHZ is None: #If there are no existing relays of this type create the dataframe
    VPHZ = pd.DataFrame(columns=relay_kf +otherfields)
else: #Remove all existing relays
    VPHZ = VPHZ[0:0].copy(deep=True)
VPHZ

,BusNum,GenID,Single,Single:1,Single:2,Single:3,Single:4


In [4]:
kf = saw.get_key_field_list('Gen')
Gens = saw.GetParametersMultipleElement('Gen', kf)

In [5]:
VPHZ = pd.concat([VPHZ, Gens], ignore_index=True)

In [6]:
VPHZ.loc[:, 'Single'] = 1.1
VPHZ.loc[:, 'Single:1'] = 60
VPHZ.loc[:, 'Single:2'] = 1.18  
VPHZ.loc[:, 'Single:3'] = 3
VPHZ.loc[:, 'Single:4'] = 0.15

In [7]:
saw.change_and_confirm_params_multiple_element(ObjectType='RelayModel_GVPHZFT', command_df=VPHZ)
nVPHZ = saw.GetParametersMultipleElement('RelayModel_GVPHZFT', relay_kf + otherfields)

In [8]:
nVPHZ

,BusNum,GenID,Single,Single:1,Single:2,Single:3,Single:4
0,1004,1,1.1,60,1.18,3,0.15
1,1006,1,1.1,60,1.18,3,0.15
2,1009,1,1.1,60,1.18,3,0.15
3,1011,1,1.1,60,1.18,3,0.15
4,1021,1,1.1,60,1.18,3,0.15
...,...,...,...,...,...,...,...
539,8152,1,1.1,60,1.18,3,0.15
540,8155,1,1.1,60,1.18,3,0.15
541,8155,2,1.1,60,1.18,3,0.15
542,8158,1,1.1,60,1.18,3,0.15


In [9]:
saw.WriteAuxFile("VperHZ_protection.aux", "","RelayModel_GVPHZFT", "ALL" )
saw.exit()